In [1]:
import dask.dataframe as dd
import numpy as np
import os

# Define source files: Jan+Feb for 2020–2024 (subset)
subset_files = [
    '/d/hpc/projects/FRI/bigdata/data/Taxi/yellow_tripdata_2020-01.parquet',
    '/d/hpc/projects/FRI/bigdata/data/Taxi/yellow_tripdata_2020-02.parquet',
    '/d/hpc/projects/FRI/bigdata/data/Taxi/yellow_tripdata_2021-01.parquet',
    '/d/hpc/projects/FRI/bigdata/data/Taxi/yellow_tripdata_2021-02.parquet',
    '/d/hpc/projects/FRI/bigdata/data/Taxi/yellow_tripdata_2022-01.parquet',
    '/d/hpc/projects/FRI/bigdata/data/Taxi/yellow_tripdata_2022-02.parquet',
    '/d/hpc/projects/FRI/bigdata/data/Taxi/yellow_tripdata_2023-01.parquet',
    '/d/hpc/projects/FRI/bigdata/data/Taxi/yellow_tripdata_2023-02.parquet',
    '/d/hpc/projects/FRI/bigdata/data/Taxi/yellow_tripdata_2024-01.parquet',
    '/d/hpc/projects/FRI/bigdata/data/Taxi/yellow_tripdata_2024-02.parquet',
]

# Final list of expected columns (all lowercase)
EXPECTED_COLUMNS = [
    'vendorid', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
    'passenger_count', 'trip_distance', 'ratecodeid', 'store_and_fwd_flag',
    'pulocationid', 'dolocationid', 'payment_type', 'fare_amount', 'extra',
    'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
    'total_amount', 'congestion_surcharge', 'airport_fee'
]

# Load and normalize each file
dfs = []

for file in subset_files:
    print(f"🔄 Reading: {file}")
    df = dd.read_parquet(file, engine='pyarrow', gather_statistics=False)

    # Normalize column names to lowercase
    df.columns = [col.lower() for col in df.columns]

    # Fill in missing columns
    for col in EXPECTED_COLUMNS:
        if col not in df.columns:
            df[col] = np.nan

    # Reorder
    df = df[EXPECTED_COLUMNS]

    dfs.append(df)

#  Concatenate everything into one Dask DataFrame
ddf = dd.concat(dfs)
print("✅ All files loaded and aligned.")

🔄 Reading: /d/hpc/projects/FRI/bigdata/data/Taxi/yellow_tripdata_2020-01.parquet
🔄 Reading: /d/hpc/projects/FRI/bigdata/data/Taxi/yellow_tripdata_2020-02.parquet
🔄 Reading: /d/hpc/projects/FRI/bigdata/data/Taxi/yellow_tripdata_2021-01.parquet
🔄 Reading: /d/hpc/projects/FRI/bigdata/data/Taxi/yellow_tripdata_2021-02.parquet
🔄 Reading: /d/hpc/projects/FRI/bigdata/data/Taxi/yellow_tripdata_2022-01.parquet
🔄 Reading: /d/hpc/projects/FRI/bigdata/data/Taxi/yellow_tripdata_2022-02.parquet
🔄 Reading: /d/hpc/projects/FRI/bigdata/data/Taxi/yellow_tripdata_2023-01.parquet
🔄 Reading: /d/hpc/projects/FRI/bigdata/data/Taxi/yellow_tripdata_2023-02.parquet
🔄 Reading: /d/hpc/projects/FRI/bigdata/data/Taxi/yellow_tripdata_2024-01.parquet
🔄 Reading: /d/hpc/projects/FRI/bigdata/data/Taxi/yellow_tripdata_2024-02.parquet
✅ All files loaded and aligned.


In [2]:
#  Add year column
OUTPUT_DIR = '/d/hpc/projects/FRI/bigdata/students/in7357/out'
ddf['year'] = dd.to_datetime(ddf['tpep_pickup_datetime']).dt.year
# Keep only rows with valid years
ddf = ddf[(ddf['year'] >= 2020) & (ddf['year'] <= 2024)]


#  Output directories
os.makedirs(os.path.join(OUTPUT_DIR, 'csv_2020'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'hdf_2020'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'csv_5yrs'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, "optimized_parquet"), exist_ok=True)

In [3]:
# STEP 1: Save 2020 subset to CSV
ddf_2020 = ddf[ddf['year'] == 2020]

ddf_2020.to_csv(f"{os.path.join(OUTPUT_DIR, 'csv_2020')}/yellow_2020_*.csv", index=False)
print(" Saved 2020 to CSV.")

 Saved 2020 to CSV.


In [6]:
# Save 2020 subset to HDF5 (via Pandas)

df_2020 = ddf_2020.compute()
for col in df_2020.select_dtypes(include="string").columns:
    df_2020[col] = df_2020[col].astype("object")

df_2020.to_hdf(f"{os.path.join(OUTPUT_DIR, 'hdf_2020')}/yellow_2020.h5", key="data", mode="w")
print(" Saved 2020 to HDF5.")

/tmp/ipykernel_3349476/4187790651.py:7: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block4_values] [items->Index(['store_and_fwd_flag', 'airport_fee'], dtype='object')]

  df_2020.to_hdf(f"{os.path.join(OUTPUT_DIR, 'hdf_2020')}/yellow_2020.h5", key="data", mode="w")


 Saved 2020 to HDF5.


In [7]:
#  STEP 3: Save all 5-year subset to CSV
ddf.to_csv(f"{OUTPUT_DIR}/csv_5yrs/yellow_5yrs_*.csv", index=False)
print(" Saved 5-year subset to CSV.")

 Saved 5-year subset to CSV.


In [8]:
#  STEP 4: Save optimized partitioned Parquet
ddf['airport_fee'] = ddf['airport_fee'].astype(str)

ddf.to_parquet(
    f"{OUTPUT_DIR}/optimized_parquet",
    engine="pyarrow",
    partition_on=["year"],
    row_group_size=1000000,
    write_index=False
)
print(" Saved to optimized Parquet partitioned by year.")

 Saved to optimized Parquet partitioned by year.
